In [2]:
import pandas as pd

In [3]:
!pwd

/Users/pranav/workspace/UIUC/Fall2020/CS_PSL/Project_3


In [8]:
split1 = pd.read_csv('split_1/train.tsv', sep='\t')

In [9]:
split1.head()

,id,sentiment,review
0,1,1,Naturally in a film who's main themes are of m...
1,4,0,Afraid of the Dark left me with the impression...
2,7,0,This has to be one of the biggest misfires eve...
3,8,0,"This is one of those movies I watched, and won..."
4,17,0,This movie was dreadful. Biblically very inacc...


In [17]:
split1['sentiment'] = split1['sentiment'].astype('int')

In [19]:
split1.dtypes

id                 int64
sentiment          int64
review            object
review_cleaned    object
dtype: object

In [15]:
## Review cleanup (this process takes a significant amount of time)
# Removing stop words, @ mentions, webpages and special characters

from nltk.corpus import stopwords # nltk.download('stopwords') before importing
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def clean(review):
    stage1 = [word for word in review.lower().split() if word not in stopwords.words('english')] # stopword removal
    return ' '.join(stage1)

In [16]:
%%time

split1['review_cleaned'] = split1['review'].apply(clean)

CPU times: user 7min 35s, sys: 1min 59s, total: 9min 34s
Wall time: 9min 36s


In [20]:
train_1x = split1['review_cleaned']
train_1y = split1['sentiment']

In [21]:
## Analysis

from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
vector = TfidfVectorizer(max_features = 1000, ngram_range = (1, 2), stop_words = 'english')
%time vector.fit(train_1x)

CPU times: user 9.55 s, sys: 214 ms, total: 9.77 s
Wall time: 9.77 s


TfidfVectorizer(max_features=1000, ngram_range=(1, 2), stop_words='english')

In [23]:
train_1x_transformed = vector.transform(train_1x)

In [24]:
train_1x_transformed

<25000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1144637 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.naive_bayes import MultinomialNB

In [28]:
nb = MultinomialNB()

In [29]:
nb.fit(train_1x_transformed, train_1y)

MultinomialNB()

In [30]:
nb.predict(train_1x_transformed[0])

array([1])

In [32]:
len(vector.vocabulary_)

1000

In [33]:
### Test it

split1_test_x = pd.read_csv('split_1/test.tsv', sep = '\t')
split1_test_y = pd.read_csv('split_1/test_y.tsv', sep = '\t')

In [34]:
split1_test_x['review_cleaned'] = split1_test_x['review'].apply(clean)

In [35]:
test_1x = vector.transform(split1_test_x['review_cleaned'])

In [37]:
test_1y = split1_test_y['sentiment']

In [38]:
pred1 = nb.predict(test_1x)

In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [41]:
print('Accuracy - {}'.format(accuracy_score(test_1y, pred1)))
print('Confusion matrix - {}'.format(confusion_matrix(test_1y, pred1)))

Accuracy - 0.83468
Confusion matrix - [[10206  2321]
 [ 1812 10661]]
